In [ ]:
'''
Test - 2.6  Harmonic Distortion
Test description - This test gives out the value the 2nd harmonic power and compare the given threshold to conclude the result hormonic power
Equipment required -
Lucid device - LS1291D
Spectrum analyzer- Agilent Technologies, E4440A (PSA Series spectrum analyzer)
what is happening in thi sscript ?
- - After initializing the device, all the parameter are define in 2nd section,
 - span and center frequency is selected according to the given table
 - reference of the spectrum must be set  5 dbm above the given power level
 -we check the given  freq and power using power_at_centerFrequency function
 - then we multiple the centre frequency with a factor of 2 to get the power at second harmonic frequency using  power_at_second_harmonics function
 - after comparing with the threshold , the final result is concluded
- A test report will be generated at the end of the script and is saved as a text file in the same directory
 '''
import numpy as np
import pyvisa as visa
import socket
import time
import re  # Set the IP Address and port

handle = 'TCPIP0::192.168.7.1::5025::SOCKET'


def send_scpi_cmd(cmd, handle):
    try:
        resourceManager = visa.ResourceManager()
        session = resourceManager.open_resource(handle)

        # Need to define the termination string
        session.write_termination = '\n'
        session.read_termination = '\n'

        session.write(cmd)
        session.close()
    except Exception as e:
        print('[!] Exception: ' + str(e))


def send_scpi_query(query, handle):
    try:
        resourceManager = visa.ResourceManager()
        session = resourceManager.open_resource(handle)

        # Need to define the termination string
        session.write_termination = '\n'
        session.read_termination = '\n'

        response = str(session.query(query))
        session.close()
        return response

    except Exception as e:
        print('[!] Exception: ' + str(e))


send_scpi_cmd('*CLS', handle)
query = '*IDN?'
temp = send_scpi_query(query, handle)
print("Connected to :", temp)
def connect_spectrum_via_lan():
    device_address = 'TCPIP::192.90.70.36::5025::SOCKET'
    try:
        rm = visa.ResourceManager()
        spectrum_analyzer = rm.open_resource(device_address)
        spectrum_analyzer.timeout = 2000
        spectrum_analyzer.write_termination = '\n'
        spectrum_analyzer.read_termination = '\n'
        print(spectrum_analyzer)
    # except visa.Error as e:
    #     print("Error while connecting: {}".format(e))
    # try:
        # Query the *IDN? command to get the identification string
        spectrum_analyzer.write('*IDN?')
        # spectrum_analyzer.timeout = 1000
        idn_response = spectrum_analyzer.read()
        spectrum_analyzer.write('*RST')
        spectrum_analyzer.write('*CLS')

        print("IDN Response: {}".format(idn_response))
    except visa.Error as e:
        print("Error reading IDN: {}".format(e))
    return spectrum_analyzer
def second_harmonic_distortion(freq_out):
    spectrum_analyzer.write(':INIT:CONT OFF')
    spectrum_analyzer.write(':TRIGger[:SEQuence]:SOURce IMM')
    spectrum_analyzer.write(':INIT:IMM')
    spectrum_analyzer.write('*TRG')
    spectrum_analyzer.write(':CONFigure:HARMonics')
    time.sleep(2)
    spectrum_analyzer.write(':INITiate:HARMonics')
    time.sleep(5)
    spectrum_analyzer.write(":FETCh:HARMonics:AMPLitude1?")
    time.sleep(2)
    spectrum_analyzer.write(':MEASure:HARMonics:AMPLitude1?')
    time.sleep(10)
    spectrum_analyzer.write(':READ:HARMonics:AMPLitude1?')
    time.sleep(2)
    resp1 = spectrum_analyzer.read()
    # spectrum_analyzer.write(":FETCh:HARMonics:AMPLitude2?")
    # time.sleep(2)
    # spectrum_analyzer.write(':MEASure:HARMonics:AMPLitude2?')
    # time.sleep(2)
    spectrum_analyzer.write(':READ:HARMonics:AMPLitude2?')
    time.sleep(10)
    resp2 = spectrum_analyzer.read()
    resp3 = float(resp2) - float(resp1)
    print('1st harmonic {1} dBm,2st HD power {0} dBc '.format(resp3,resp1))
    return resp3
def power_at_second_harmonics(center_frequency):
    second_freq = 2*center_frequency
    #spectrum_analyzer.write(':CALC:MARK1:STAT OFF')
    spectrum_analyzer.write('FREQ:CENT {0} MHz'.format(second_freq))
    time.sleep(2)
    spectrum_analyzer.write('DISP:WIND:TRAC:Y:RLEV -20 dbm'.format(ref_level))
    time.sleep(2)
    spectrum_analyzer.write(':CALC:MARK1:STAT ON;:CALC:MARK1:X {} MHz'.format(second_freq))

    time.sleep(1)
    spectrum_analyzer.write('CALC:MARK1:X?')
    time.sleep(1)
    resp = spectrum_analyzer.read()
    time.sleep(1)
    freq_out = float(resp) / 1e6
    time.sleep(1)
    spectrum_analyzer.write('CALC:MARK1:Y?')
    time.sleep(1)
    power_max = spectrum_analyzer.read()
    time.sleep(1)
    print('power at second harmonic ={0},at freq= {1},which should be around {2}'.format(power_max,freq_out,second_freq))
    return power_max,freq_out

def power_at_centerFrequency(spectrum_analyzer):
    spectrum_analyzer.write(':CALC:MARK1:STAT ON;:CALC:MARK1:MAX')
    time.sleep(1)
    spectrum_analyzer.write('CALC:MARK1:X?')
    time.sleep(1)
    resp = spectrum_analyzer.read()
    time.sleep(1)
    freq_out = float(resp) / 1e6
    #print('centre frequency = {} MHz'.format(freq_out))
    time.sleep(1)
    spectrum_analyzer.write('CALC:MARK1:Y?')
    time.sleep(1)
    power_max = spectrum_analyzer.read()
    time.sleep(1)
    print('power ={1} dBm at frequency = {0} MHz'.format(freq_out, power_max))
    return power_max,freq_out

spectrum_analyzer = connect_spectrum_via_lan()
spectrum_analyzer.write(':INIT:REST')
print('Initialized')
test_success = True
####################################################################
# Input Parameters (madify this for 40G accordingly)
frequency_list =list(np.arange(10,1000,100))+list(np.arange(1100,8000,1000))+list(np.arange(8100,12000,1000)) #+list(np.arange(20000,40000,100)
power_list= [5]
table_i = 0
###Test Report Table parameters
serial_no_tb = np.zeros(len(frequency_list)*len(power_list))
input_list_tb = np.zeros(len(frequency_list)*len(power_list))
power_list_tb = np.zeros(len(frequency_list)*len(power_list))
power_out_tb=np.zeros(len(frequency_list)*len(power_list))
freq_out_tb =np.zeros(len(frequency_list)*len(power_list))
output_list_tb = np.zeros(len(frequency_list)*len(power_list))
output_status_tb= []# To store the output frequency from spectrum analyzer (optional step)
error_limit_tb = -40+np.zeros(len(frequency_list)*len(power_list)) # error limit in dBm
###########################################
###########################################
for i in range(len(frequency_list)):
    center_frequency = frequency_list[i]
    for power_in_dBm in power_list:
        ref_level = power_in_dBm + 5  # reference level on the spectrum must be 5dBm above the signal power level
        threshold = error_limit_tb[table_i] + power_in_dBm
        send_scpi_cmd(':FREQuency {0}e6'.format(center_frequency), handle)
        send_scpi_cmd(':POWer {0}'.format(power_in_dBm), handle)
        send_scpi_cmd(':OUTPut ON', handle)
        time.sleep(2)
        spectrum_analyzer.write(':INIT:REST')
        time.sleep(2)
        spectrum_analyzer.write('DISP:WIND:TRAC:Y:RLEV {} dbm'.format(ref_level))
        time.sleep(2)
        spectrum_analyzer.write('FREQ:CENT {0} MHz'.format(center_frequency))
        time.sleep(2)
        spectrum_analyzer.write('FREQ:SPAN 20 MHz'.format(center_frequency / 10))
        time.sleep(2)
        power_out_tb[table_i],freq_out_tb[table_i]=power_at_centerFrequency(spectrum_analyzer) # function to read power from spectrum analyzer
        output_list_tb[table_i],second_freq=power_at_second_harmonics(center_frequency)
######################################################
        #output_list_tb[table_i]= second_harmonic_distortion(freq_out_tb[table_i])
        serial_no_tb[table_i] = table_i + 1
        input_list_tb[table_i] = center_frequency
        power_list_tb[table_i] = power_in_dBm
        if output_list_tb[table_i] <threshold:
            output_status_tb.append("pass")
            print("TEST pass for frequency {}MHz".format(center_frequency))
        else:
            test_success = False
            output_status_tb.append("fail")
            print("TEST FAIL for frequency {}MHz".format(center_frequency))

        table_i =table_i +1
if test_success:
    print("Test successed!")
else:
    print("Test failed ")
    # disconnecting the instruments
spectrum_analyzer.close()

In [1]:
##########################################
#Test report
from tabulate import tabulate
# Sample data for the table
data = list(zip(serial_no_tb,input_list_tb,power_list_tb,error_limit_tb,freq_out_tb,power_out_tb,output_list_tb,output_status_tb))
# Creating the table
table = tabulate(data, headers=["Test step","Input frequency","Power","Error Limits (in dBc)","Output frequency","Output Power","Max HD Spectrum reading(2nd harmonic)","PASS/FAIL"], tablefmt="grid")
# Displaying the table
print(table)
# Writing the table to a text file
with open("Harmonic distortion test report.txt", "w") as f:
    f.write(table)

print("Table has been written to table.txt")

## To do later - work on the direct method to get the harmonics (function already created as "second_harmonic_distortion") rather then positioning the marker 2 at twice of the centre frequency and getting the power of it to verify the 2nd harmonic distortion

Connected to : Tabor Electronics,LSX2091D,20240101,1.0.2
TCPIPSocket at TCPIP0::192.90.70.36::5025::SOCKET
IDN Response: Agilent Technologies, E4440A, MY44303333, A.09.10
Initialized
power =+4.54300E+00 dBm at frequency = 10.0 MHz
power at second harmonic =-4.20650E+01,at freq= 20.0,which should be around 20
TEST pass for frequency 10MHz
power =+5.14300E+00 dBm at frequency = 110.0 MHz
power at second harmonic =-4.29730E+01,at freq= 220.0,which should be around 220
TEST pass for frequency 110MHz
power =+4.76900E+00 dBm at frequency = 210.0 MHz
power at second harmonic =-4.42280E+01,at freq= 420.0,which should be around 420
TEST pass for frequency 210MHz
power =+4.85100E+00 dBm at frequency = 310.0 MHz
power at second harmonic =-4.97080E+01,at freq= 620.0,which should be around 620
TEST pass for frequency 310MHz
power =+4.94700E+00 dBm at frequency = 410.0 MHz
power at second harmonic =-4.85300E+01,at freq= 820.0,which should be around 820
TEST pass for frequency 410MHz
power =+5.40100E